# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

### Get data, option 1: Kaggle API

#### Sign up for Kaggle and get an API token
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Follow these instructions](https://github.com/Kaggle/kaggle-api#api-credentials) to create a Kaggle “API Token” and download your `kaggle.json` file. If you are using Anaconda, put the file in the directory specified in the instructions.

_This will enable you to download data directly from Kaggle. If you run into problems, don’t worry — I’ll give you an easy alternative way to download today’s data, so you can still follow along with the lecture hands-on. And then we’ll help you through the Kaggle process after the lecture._

#### Put `kaggle.json` in the correct location

- ***If you're using Anaconda,*** put the file in the directory specified in the [instructions](https://github.com/Kaggle/kaggle-api#api-credentials).

- ***If you're using Google Colab,*** upload the file to your Google Drive, and run this cell:

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


#### Install the Kaggle API package and use it to get the data

You also have to join the Titanic competition to have access to the data

In [2]:
!pip install kaggle

In [3]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 48.3MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.6MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.69MB/s]


### Get data, option 2: Download from the competition page
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Go to the Titanic competition page](https://www.kaggle.com/c/titanic) to download the [data](https://www.kaggle.com/c/titanic/data).

### Get data, option 3: Use Seaborn

```
import seaborn as sns
train = sns.load_dataset('titanic')
```

But Seaborn's version of the Titanic dataset is not identical to Kaggle's version, as we'll see during this lesson!

### Read data

In [13]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')  # Unlabeled, for Kaggle submission

print(train_df.shape)
print(test_df.shape)

(891, 12)
(418, 11)


In [14]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [15]:
test_df.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [16]:
train_df.describe(include='number')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### How would we try to do this with linear regression?

https://scikit-learn.org/stable/modules/impute.html

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
features = ['Pclass','Age','Fare']
target = 'Survived'
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]

imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
#transform applies trains transformation without leakage from test

lin_reg = LinearRegression()
lin_reg.fit(X_train_imputed,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [18]:
import numpy as np
test_case = np.array([[1, 5, 500]])
lin_reg.predict(test_case)

array([1.19207871])

### How would we do this with Logistic Regression?

In [23]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='liblinear')
log_reg.fit(X_train_imputed,y_train)
log_reg.predict(test_case)

array([1])

In [24]:
log_reg.predict_proba(test_case)

array([[0.02081599, 0.97918401]])

### How accurate is the Logistic Regression?

In [25]:
log_reg.score(X_train_imputed, y_train)


0.7048260381593715

In [27]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, X_train_imputed, y_train, cv=10)
print('Cross Val scores:',scores)

Cross Val scores: [0.63333333 0.62222222 0.70786517 0.69662921 0.68539326 0.70786517
 0.73033708 0.75280899 0.71910112 0.71590909]


In [28]:
scores = pd.Series(scores)
scores.min(),scores.mean(),scores.max()

(0.6222222222222222, 0.6971464646464647, 0.7528089887640449)

### What's the math for the Logistic Regression?

https://en.wikipedia.org/wiki/Logistic_function

https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study

## Feature Engineering

Get the [Category Encoder](http://contrib.scikit-learn.org/categorical-encoding/) library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [132]:
!pip install category_encoders

In [0]:
(train_df['Sex'] == 'male') & (train_df['Age'] >= 16)

In [0]:
def make_features()

## Assignment: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

### Get and unzip the data

#### Google Colab

In [133]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-05-06 21:38:55--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip.1’

fma_metadata.zip.1  100%[===================>] 341.81M  26.1MB/s    in 14s     

2019-05-06 21:39:10 (24.6 MB/s) - ‘fma_metadata.zip.1’ saved [358412441/358412441]

Archive:  fma_metadata.zip
replace fma_metadata/README.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/checksums? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/not_found.pickle? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/raw_genres.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/raw_albums.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fma_metadata/raw_artists

#### Windows
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [7zip](https://www.7-zip.org/download.html) to unzip it


#### Mac
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [p7zip](https://superuser.com/a/626731) to unzip it

### Look at first 3 lines of raw file

In [0]:
!head -n 3 fma_metadata/tracks.csv

### Read with pandas
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [0]:
pd.set_option('display.max_columns',500)
df = pd.read_csv('fma_metadata/tracks.csv')
df.head(2)

In [0]:
df.iloc[0]['Unnamed: 0']='track_id'
df['Unnamed: 0']
new_header=df.iloc[0]
df=df[1:]
df.columns = new_header
df=df.drop([1]).reset_index(drop=True)
df.head()


In [122]:
df.shape

(106574, 53)

In [136]:
#delete dupe columns like date_created 
df = df.loc[:,~df.columns.duplicated()]
df.head(1)

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,latitude,location,longitude,members,name,related_projects,website,wikipedia_page,split,subset,bit_rate,composer,date_recorded,duration,genre_top,genres,genres_all,interest,language_code,license,lyricist,number,publisher
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,NaN,2008-11-26 00:00:00,168,Hip-Hop,[21],[21],4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,NaN,3,NaN


In [0]:
df['date_created'] = pd.to_datetime(df['date_created'], infer_datetime_format=True)
df['date_released'] = pd.to_datetime(df['date_released'], infer_datetime_format=True)
df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True)
df['active_year_begin'] = pd.to_datetime(df['active_year_begin'], infer_datetime_format=True)
df['date_end'] = pd.to_datetime(df['active_year_end'], infer_datetime_format=True)



In [0]:
'''to int: 'track_id', 'comments', 'favorites','id', 'listens', 'tracks',
'latitude', 'longitude', 'number','duration'
'''
#change to time span: duration
#change nan to n/a: 'engineer', 'producer', 'publisher'
#bring up rows for type:nan
#remove html for information and bio
'''useless?: members, bio, information, maybe related_projects,
website, wikipedia_page, genres, genres_all
'''
#meta: split, subset
df[['track_id', 'comments', 'favorites','id', 'listens', 'tracks',
'latitude', 'longitude', 'number','duration']]=df[['track_id',
'comments', 'favorites','id', 'listens', 'tracks',
'latitude', 'longitude', 'number','duration']].astype(float)

In [0]:
import numpy as np
df[['engineer', 'producer', 'publisher','type']]= df[['engineer', 'producer',
    'publisher','type']].fillna('N/A')
df.drop(columns=['members', 'bio', 'information', 'related_projects',
'website', 'wikipedia_page', 'genres_all'],inplace=True)


In [0]:
df.lyricist = df.lyricist.fillna('none')

In [141]:
df.genre_top.describe()

count     49598
unique       16
top        Rock
freq      14182
Name: genre_top, dtype: object

Ok let's try to see whether or not it is rock.
Some ideas are location, maybe including long and lat to get locations (even though I don't know how to do that.)

Genres itself would probably be a good predictor. It has a number, so maybe that is the corresponding number for genre_top?

listens, duration, interest, favorites? idk about duration, but maybe more of the others would befit the top genre.

liscense? perhaps genres share a similar liscense?

publishers and producers probably would be good. I imagine that these sort of guys stick to what they know.

In [142]:
df.genre_top.unique()

array(['Hip-Hop', 'Pop', nan, 'Rock', 'Experimental', 'Folk', 'Jazz',
       'Electronic', 'Spoken', 'International', 'Soul-RnB', 'Blues',
       'Country', 'Classical', 'Old-Time / Historic', 'Instrumental',
       'Easy Listening'], dtype=object)

In [0]:
#need an is_rock column
df['is_rock'] =df['genre_top'].map({'Rock':1}).fillna(0).astype(int)

In [169]:
df.dtypes

0
track_id                    float64
comments                    float64
date_created         datetime64[ns]
date_released        datetime64[ns]
engineer                     object
favorites                   float64
id                          float64
listens                     float64
producer                     object
tags                         object
title                        object
tracks                      float64
type                         object
active_year_begin    datetime64[ns]
active_year_end              object
associated_labels            object
latitude                    float64
location                     object
longitude                   float64
name                         object
split                        object
subset                       object
bit_rate                     object
composer                     object
date_recorded        datetime64[ns]
duration                    float64
genre_top                    object
genres                    

### Fit Logistic Regression!

In [0]:
from sklearn.model_selection import train_test_split
features = ['number', 'duration']
target = 'is_rock'
#X_train, X_test, y_train, y_test = train_test_split(df[features],df[target], train_size=0.50, test_size=0.50)

In [171]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='liblinear')
log_reg.fit(df[features],df[target])



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [172]:
log_reg.score(df[features],df[target])

0.8669281438249479

This dataset is bigger than many you've worked with so far, and while it should fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting, or [downcasting numeric datatypes](https://www.dataquest.io/blog/pandas-big-data/).
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.